# Segmentace obrazu a měření rozměrů
Základní znalostí při zpracování obrazu je nejenom ideálně nasnímat obraz, ale hlavně ze získaného obrazu dostat použitelné informace. Extrakce objektů z obrazu (oddělení objektů od pozadí) se nazívá segmentace. Nejčastěji se využívá metod prahování, hranové detekce nebo kontur ([viz link](https://circuitdigest.com/tutorial/image-segmentation-using-opencv)).

Když už se podaří získat z obrazu objekty, je dále potřeba změřit jejich rozměry - a to nejen v pixelech, ale hlavně reálné rozměry v cm nebo dokonce v menších jednotkách. 

Na cvičení se bude využívat umělý obrázek s objekty jejichž rozměry bude třeba automatizovaně získat.

![](images/index.jpg)

### Import knihoven a konfigurace

In [1]:
%run ../svz.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:


První část
- [`show_images(...)`](../svz.ipynb#show_functions)
- [`show_images(...)`](../svz.ipynb#show_functions)
- [`load_image(...)`](../svz.ipynb#load_save_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)

Druhá část
- [`copy_to(...)`](../svz.ipynb#measuring_funtions)
- [`midpoint(...)`](../svz.ipynb#measuring_funtions)
- [`order_points(...)`](../svz.ipynb#measuring_funtions)
- [`rotate_image(...)`](../svz.ipynb#measuring_funtions)
- [`draw_rotated_text(...)`](../svz.ipynb#measuring_funtions)
- [`draw_real_sizes(...)`](../svz.ipynb#measuring_funtions)

---

### Úkol 1

Úkol je zaměřen na segmentace obrazů, získání kontur objektů a měření poměru rozměrů v cm a pixelech.

**1) Načtěte a zkontrolujte zaznamenaný obrázek vzoru pomocí kamery. Nadefinujte proměné s šířkou a výškou referečního objektu.**

Obrázek nastavte tak, aby byl ideálně záběr pouze na něj. *HINT: Je možné pomoci si oříznutím obrázku (crop).*

In [2]:
img_patterns = load_image('patterns_photo_basic.jpg') ###
img_cropped = crop(img_patterns, 280, 300, 3900, 2500)
show_images(img_cropped)

In [3]:
ref_width_real = 4
ref_height_real = 8

**2) Využijte funkci [cv2.inRange()](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981) k získání masky referenčního obdélníku.**

Funkce kromě zdrojového obrázku přijímá dolní a horní mez hodnoty pixelů ve formátu BGR (pozor celé OpenCV funguje v režimu BGR, ne RGB). Tedy dvě trojice (tuple) ve formátu (b, g, r), kde 0 <= b, g, r <= 255. Pixely s hodnotou >= než dolní mez a <= než horní mez budou mít ve výsledné masce hodnotu 255, ostatní 0. 

Dolní a horní mez naleznete experimentálně.

In [4]:
ref_mask = cv2.inRange(img_cropped, (0,70,120), (100,235, 255)) ### tuples
show_images(ref_mask)

**3) Naleznete konturu která pokrývá nevětší plochu, otestujte aproximace kontury na obdélník.**

Výsledná maska by měla ideálně obsahovat pouze referenční obdelník, v obrazu se ale obecně může vyskytovat i šum. 
Nyní je nutné získat pozici referenčního obdelníku v obraze masky. 
Nejjednoduším způsobem je vyhledání kontur (obrysů) v obraze pomocí [cv2.findContours()](https://docs.opencv.org/3.3.1/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a). Vyhledání kontur funguje jen na binárním černobílém obraze, proto jsme nejdříve museli využít cv2.inRange() k získání masky.

Konturu si lze představit jako křivku spojující několik bodů kolem obrysu souvislého objektu. Funkce cv2.findContours() příjímá navíc dva parametry, contour retrieval mode a contour approximation method. Těmito parametry se ve většině případů nemusíme zabývat. Zjednodušeně je tedy nastavíme `mode=cv2.RETR_LIST` a `method=cv2.CHAIN_APPROX_SIMPLE`. Pokud by vás přeci jen zajímaly, můžete si o nich přečíst [zde](https://docs.opencv.org/3.3.1/d4/d73/tutorial_py_contours_begin.html)

Vzhledem k šumu, který se na každém snímku vyskytuje, prakticky nikdy nenajdeme konturu pouze jednu. Je tedy nutné následně provést filtrování. V našem případě si vystačíme s výběrem kontury, která má největší plochu.

Posledním krokem je validace našeho postupu a vizualizace nalezené kontury pomocí funkce [cv2.drawContours()](https://docs.opencv.org/3.3.1/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc).
                                                                                

In [5]:
contours, _  = cv2.findContours(ref_mask, mode=cv2.RETR_LIST ,method=cv2.CHAIN_APPROX_SIMPLE) 
print(f'Found {len(contours)} contours.')

# pick only the contour with the biggest area
contour_biggest = max(contours, key=cv2.contourArea)
# print(f'Biggest contour area: {cv2.contourArea(contour_biggest)}, coordinates:\n {contour_biggest.reshape((-1, 2)).tolist()}')
contour_drawn = cv2.drawContours(img_cropped.copy(), [contour_biggest], -1, color=(255, 0, 0), thickness=5)
show_images(contour_drawn)

Found 975 contours.


Z výsledných souřadnic vidíme, že kontura netvoří přesný obdelník, proto ji musíme obdelníkem aproximovat. To lze pomocí funkce [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Návratovou hodnotou této funkce je tuple - (střed obdélníku (x,y), (výška, šířka), úhel rotace obdélníku). Jedna otázka zní, co je výška a co je šířka? Musíte to vždycky kontrolovat okem.

Pomocná funkce [cv2.boxPoints()](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#gaf78d467e024b4d7936cf9397185d2f5c) převádí nalezený obdelník z formátu (střed obdélníku (x,y), (šířka, výška), úhel rotace obdélníku) na 4 rohové body obdélníku ve formátu (x, y). To se může hodit např. k vizualizaci (lze použít jako vstup pro cv2.drawContours()). Pozor, pořadí vrácených bodů není zaručeno.

In [6]:
rect = cv2.minAreaRect(contour_biggest) ###
print(f'Rect tuple: {rect}')
print(f'(cx, cy)={rect[0]}, (width, height)={rect[1]}, angle={rect[2]}')
print(f'Rect points: {cv2.boxPoints(rect).tolist()}')

Rect tuple: ((365.51617431640625, 1333.271240234375), (544.0951538085938, 1099.3265380859375), -8.586238861083984)
(cx, cy)=(365.51617431640625, 1333.271240234375), (width, height)=(544.0951538085938, 1099.3265380859375), angle=-8.586238861083984
Rect points: [[178.5811767578125, 1917.3900146484375], [14.4541015625, 830.3846435546875], [552.451171875, 749.1524658203125], [716.5782470703125, 1836.1578369140625]]


**5) Nalezenou šířku referenčního obdelníku v pixelech můžeme konečně využít k získání poměru skutečné šířky obdélníku v reálných jednotkách a pixelové šířky obdélníku v obraze. Tento poměr budeme následně potřebovat k výpočtu rozměrů ostatních neznámých objektů.**

In [7]:
ref_width_image, ref_height_image =  rect[1]
real_image_ratio = ref_width_real / ref_width_image
print(f'Ratio between real width and image width: {real_image_ratio}')

Ratio between real width and image width: 0.007351655261033905


---

### Úkol 2

Úkol 1 máme za sebou a už víme poměr mezi rozměrem v cm a pixelech, tím pádem už můžeme naměřit rozměry ostatních objektů na obrázku s neznámými rozměry.

**1) Opět využijte funkci [cv2.inRange()](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981), tentokrát k získání masky objektů s neznámými skutečnými rozměry.**

**2) Nalezněte v nové masce snímku kontury stejně jako v případě referenčního objektu.**

**3) Následně je provedeno filtrování kontur podle jejich obsahu. Prahovou hodnotu obsahu v pixelech (threshold) je nutno zvolit experimentálně.**

In [8]:


patterns_mask = cv2.inRange(img_cropped, (30,90,30), (190,200, 120)) ### tuples

show_images(patterns_mask)

In [9]:
contours, _  = cv2.findContours(patterns_mask, mode=cv2.RETR_LIST ,method=cv2.CHAIN_APPROX_SIMPLE) 
print(f'Found {len(contours)} contours.')

# Filter out noise
threshold = 10000 ###
contours =  [c for c in contours if cv2.contourArea(c) > threshold]
print(f'After filtering, {len(contours)} contours remained.')

contour_drawn = cv2.drawContours(img_cropped.copy(), contours, -1, color=(255, 0, 0 ), thickness=10)
show_images(contour_drawn)

# Sort contours by area. Just for better debugging.
contours.sort(key=cv2.contourArea, reverse=True)

Found 8709 contours.
After filtering, 4 contours remained.


**4) Nalezněte skutečné rozměry objektu**

Nyní potřebujeme pro jednotlivé kontury zjistit jejich skutečné rozměry, které chceme vizualizovat do výsledného obrázku.

Pro každou konturu tedy získáme jeji obdélníkovou aproximaci pomocí [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Následně můžeme vypočítat skutečnou šířku a výšku objektu, díky předešlému vypočítanému poměru mezi skutečnou šířkou a pixelovou šířkou. Posledním krokem je volání funkce `draw_real_sizes()`, která se do vstupního obrázku pokusí vykreslit rozměry nalezeného objektu.

In [10]:
# create a copy of original image
sizes_drawn = img_cropped.copy()

for c in contours:
    rect = cv2.minAreaRect(c)
    shape_height, shape_width = rect[1]
    real_width = real_image_ratio * shape_width
    real_height = real_image_ratio * shape_height
    
    cv2.drawContours(sizes_drawn, [c], -1, color=(255, 0, 0 ), thickness=5)
    sizes_drawn = draw_real_sizes(sizes_drawn, rect, real_height, real_width,
                                  lbl_size_scale=.7, lbl_color=(0, 0, 255), lbl_thickness=1) ###

show_images(sizes_drawn)

### Úkol

Zadání tohoto úkolu prověřuje vaše pochopení látky probírané na cvičení. Jeho cílem je implementace __výpočtu nejkratší vzdálenosti mezi obdélníky__ na zadaném zdrojovém obrázku. Úkol má první __základní část s nápovědami__ a druhou __dobrovolnou část__ s bonusovými rozšířeními. __Maximální počet bodů za základní část je 5__, z bonusové části je možné získat až __2 prémiové__. Podrobnosti o bonusech jsou na konci notebooku. Pokud si s něčím nebudete dlouho vědět rady, napište email a domluvíme konzultaci.


Nyní uvažujme pouze základní variantu. Na začátek si je důležité uvědomit, že se __nejedná o vzdálenost mezi středy obdélníků__, nýbrž o nejkratší možné vzdálenosti hran objektů viz. [obrázek](text_imgs/patterns_lengths_example.JPG). Nezoufejte nyní, pokud nevíte jak vzdálenosti vypočítat, nápovědy v notebooku vám pomohou s řešením. Nemusíte se však striktně držet postupu, jakákoliv __individualita je vítaná__. K funkčnímu řešení se dá dobrat různými způsoby.

![](text_imgs/patterns_lengths_example.JPG)


Možná se ptáte, k čemu je to vlastně dobré. Motivace je ale jednoduchá - jedná se totiž o celkem běžně řešený problém v praxi. Navíc, v knihovně OpenCV pro to __neexistuje nativní implementace__, takže pokud zůstanete v oboru, může se vám to v budoucnu hodit. Představte si nyní, že vytváříte aplikaci, která má za úkol automatizovaně hrát hru ve které je nutné předcházet kolizím objektů ve scéně. Objekty umíte lokalizovat a ohraničit (nejčastěji obdélníkem), a mohou být různě velké a natočené. Jak nyní zjistit, jak jsou od sebe objekty daleko? V případě, že nebudou objekty (tedy ani obdélníky) natočené, lze vzdálenost vypočíst se znalostí středoškolské geometrie. V reálných situacích se tohle ale moc často nestává. Objekty jsou většinou libovolně natočené a tím se řešení trochu komplikuje. Stále si ale vystačíme se znalostí analytické geometrie ze střední školy, jen bude potřeba trochu programování navíc. Mimo kolize ve hře si můžeme představit využití i např. v aplikaci pro sledování pohybujících se objektů nebo monitorování průmyslových procesů (výrobní linka, spojování materiálů, atd.). 

Abychom si zbytečně nekomplikovali život, budeme řešení implementovat pouze na jednoduchém statickém snímku s několika obdélníky. __Zdrojový snímek základní verze je__ [patterns_photo_basic.jpg](patterns_photo_basic.jpg). Využijeme naší znalost ze cvičení a budou nás __zajímat nejkratší vzdálenosti v centimetrech__, nikoliv v pixelech.
![](text_imgs/patterns_thumb.jpg)

Předpokladem tohoto úkolu je, že __máte vypracovaný notebook z 3. cvičení__. Budeme totiž znovu potřebovat segmentované kontury všech objektů, a vzhledem k tomu, že už máme nějakou práci hotovou, byla by škoda na ni nenavázat. Využijeme tedy vypracovaný notebook ze cvičení, upravíme pouze barevné intervaly ve funkcích `inRange()` tak, aby dávaly smysl pro nový zdrojový obrázek. 

Na konec vyplněného notebooku ze cvičení rafinovaně __napojíme všechny buňky tohoto notebooku__. Nejjednoduší způsob jak to navázat je: 
* Pro jistotu si zálohovat původní notebook (pokud ho nemáte na gitu, fakt to udělejte).
* Zkopírovat si potřebné zdrojové obrázky do stejné složky.
* Otevřít si tento notebook a notebook ze cvičení v textovém editoru.
* Všimněte si poměrně jednoduché struktury souboru ipynb notebooků. Jedná se o klasický formát JSON. Všechny buňky souboru se nachází v `{ "cells": [....] }`.
* Obsah struktury `"cells"` zkopírujeme a vložíme na konec seznamu `"cells"` v souboru ze cvičení. Dejte si pozor ať vám sedí čárky a uzavírací závorky "]".

### Základní část

In [11]:
import itertools
import math
import cv2

V řešení ze cvičení už máme hotové segmentace referenčního i ostatních objektů uložené formou kontur. 
Všechny tyto kontury se vyplatí přemístit do jedné struktury, neboť se s ní následně bude lépe pracovat.

__1) Přesuňte všechny kontury do jedné struktury (např. list).__

In [12]:
all_contours = [c for c in contours]
all_contours.append(contour_biggest)

__2) Pomocí funkce `len()` ověřte, že jich máte správný počet.__

In [13]:
len(all_contours)

5

__3) Uložte si hodnoty všech možných indexů předchozího listu do nového listu. Může se vám k tomu hodit funkce `range()` + `list()`. Ideálně využijte i předchozí informaci o počtu kontur.__

In [14]:
indexes = list(range(len(all_contours)))

__4) Využijte funkci `itertools.combinations()` k získání všech možných dvojic kombinací indexů kontur. Pro jistotu si je vypiště.__

In [15]:
all_indexes = list(itertools.combinations(tuple(indexes) , 2))
print(all_indexes)

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


Nyní už to konečně začne být trochu zajímavé. Zbývá __projít všechny možné kombinace kontur, aproximovat je obdélníkem
vypočítat vzdálenost mezi všemi obdélníky, a výsledek vhodně vizualizovat v reálných jednotkách__. 
To je na jednu Jupyter buňku až moc operací, lepší řešení tedy bude tyto kroky rozpadnout do několika funkcí.

Způsobů výpočtu nejkratší vzdáleností mezi obdelníky je více. Můžete samozřejmě využít jakékoliv řešení budete chtít (za předpokladu korektnosti výsledků). V tomto postupu budem však uvažovat pouze jedno a to, které se nám zdálo nejjednoduší na pochopení. 

Pro zjedodnušení celé úlohy __není nutné uvažovat žádné složité speciální případy__ - nejsme Progtest. Nemusíte tedy uvažovat, že se tvary jakkoliv překrývají. Předpokládáme, že vzdálenost je vždy > 0. S tím také souvisí pozdější detaily jako __neřešení případů bodu ležícího na úsečce, úsečky se protínají, atd__.

Začneme tedy prvním stavebním kamenem našeho algoritmu, tedy __funkcí, která vypočte vzdálenost bodu od úsečky v pixelech__. Tato funkce je jádrem celého našeho řešení, je tedy nutné si ověřit, že vrací správné hodnoty.

Pokud vůbec netušíte, jak byste takovou věc implementovali, hledejte klíčová slova `"distance between line segment and point"` a inspirujte se. Nezapomeňte však __uvést zdroj__ + případne okomentování. __Dejte si pozor, že line segment != line__, pracujeme pouze s úsečkami.

__5) Doplňte funkci pro výpočet vzdálenosti mezi úsečkou a bodem.__

In [16]:
def line_segment_to_point_dist(l_pt1, l_pt2, dst_pt):
    """
    Zdroj 
    `https://math.stackexchange.com/questions/322831/determing-the-distance-from-a-line-segment-to-a-point-in-3-space/322836`
    Nejkratší vzdálenost mezi přímkou a bodem je kolmice. Pokud však máme jen úsečku je to trochu složitější
    Potřebujeme spočítat místo, kde by byla kolmice. Pokud je to místo mimo úsečku, tak v závislosti jestli je to 
    kladné nebo záporné, tak spočítáme jednoduše pomocí pythagorovy věty. Pokud je to v úsečce, použijeme vzorec pro přímku
    """
    u = np.array((l_pt2[0] - l_pt1[0], l_pt2[1] - l_pt1[1]))
    v = np.array((dst_pt[0] - l_pt1[0], dst_pt[1] - l_pt1[1]))
    tmp = np.inner(u, v) / abs(np.inner(u, u))
    # print("tmp is", tmp)
    if tmp > 1:
        return ((l_pt2[0]-dst_pt[0])**2 + (l_pt2[1]-dst_pt[1])**2)**.5
    elif tmp < 0:
        return ((l_pt1[0]-dst_pt[0])**2 + (l_pt1[1]-dst_pt[1])**2)**.5
    elif 0 <= tmp <=1:
        p = l_pt1[0] + tmp*u[0], l_pt1[1] + tmp*u[1]
        return ((p[0]-dst_pt[0])**2 + (p[1]-dst_pt[1])**2)**.5
    raise ValueError()

__6) Otestujte, že funkce vrací správné výsledky a dopište alespoň dva další asserty.__

In [17]:
assert line_segment_to_point_dist((1,3), (3,4), (4,4)) ==  1
assert line_segment_to_point_dist((1,3), (3,4), (-1,3)) ==  2
assert line_segment_to_point_dist((5,7), (5,2), (3,4)) ==  2
assert line_segment_to_point_dist((5,7), (5,1), (3,4)) ==  2

Nyní potřebujeme další stavební blok, a to nejkratší vzdálenost dvou úseček. K tomu využijeme už hotovou funkci `line_segment_to_point_dist()`. Pokud vás nenapadá jak, zkuste si to nakreslit na papír.
Nezapomeňte, že __nemusíme uvažovat speciální případy__ (např. to, že se protínají). S využítím funkce `min()` je to dokonce pouze na jeden řádek kódu.

__7) Doplňte funkci pro výpočet vzdálenosti mezi dvěmi úsečkami.__

In [18]:
def line_segments_dist(l1_pt1, l1_pt2, l2_pt1, l2_pt2):
    return min(
        line_segment_to_point_dist(l1_pt1, l1_pt2, l2_pt1),
        line_segment_to_point_dist(l1_pt1, l1_pt2, l2_pt2),
        line_segment_to_point_dist(l2_pt1, l2_pt2, l1_pt1),
        line_segment_to_point_dist(l2_pt1, l2_pt2, l1_pt2),
    )

__8) Otestujte, že funkce vrací správné výsledky a dopište alespoň dva další asserty.__

In [19]:
#print(line_segments_dist((1,5),(6,5), (7,7), (7,5)))
assert line_segments_dist((1,3), (3,4), (5,7), (5,1)) ==  2
assert line_segments_dist((1,5),(6,5), (7,7), (7,5)) == 1

Když už umíme vypočítat nejkratší vzdálenost dvou úseček, máme vše potřebné pro výpočet nejkratší vzdálenosti mezi dvěma obdélníky. Stačí si jen uvědomit, že obdélník je složen ze čtyř spojených úseček. 

Typ vstupních parametrů funkce `rect_dist` je na vás. Nezapomeňte však, že `cv2.minAreaRect()` nevrací přímo souřadnice rohových bodů. Připomeňte si `cv2.boxPoints()` a `order_points()`.

__9) Doplňte funkci pro výpočet vzdálenosti mezi dvěma obdélníky.__

In [20]:
def rect_dist(r1_pts, r2_pts):
    r1_c = itertools.combinations(r1_pts, 2)
    r2_c = itertools.combinations(r2_pts, 2)
    return min([line_segments_dist(x[0], x[1], y[0], y[1]) for x in r1_c for y in r2_c])

__10) Otestujte, že funkce vrací správné výsledky a dopište alespoň dva další asserty.__

In [21]:
assert rect_dist(((1,5), (6,5), (6,3), (1,3)), ((7,7), (9,7), (9, 5), (7, 5))) == 1
assert rect_dist(((1,5), (6,5), (6,3), (1,3)), ((4,10), (6, 8), (4, 6), (2,8))) == 1

Blížíme se do finále. Úplně v první buňce tohoto úkolu bychom měli mít vytvořený list se všemi konturami a list všech dvojic kombinací indexů kontur. Kontury vhodně převeďte tak, aby je bylo možné předat do definované funkce `rect_dist()` a tím získat vzdálenost mezi dvěma obdélníky v pixelech. Následně přepočítejte pixely na skutečné jednotky a vypiště nalezené vzdálenosti ve vhodném a kontrolovatelném formátu (např.: 1 <-> 3: 2.2 cm). 

__11) Vypočítejte vzdálenosti mezi všemi obdélníky ve snímku, vypiště vzdálenosti ve vhodném formátu v centimetrech.__

In [22]:
tuple(all_contours[0][0][0])
rect = cv2.minAreaRect(all_contours[0])
print(f'Rect tuple: {rect}')
print(f'(cx, cy)={rect[0]}, (width, height)={rect[1]}, angle={rect[2]}')
print(f'Rect points: {cv2.boxPoints(rect).tolist()}')
cv2.boxPoints(rect)

Rect tuple: ((3037.648681640625, 1063.90771484375), (1914.8492431640625, 550.8592529296875), -82.80413055419922)
(cx, cy)=(3037.648681640625, 1063.90771484375), (width, height)=(1914.8492431640625, 550.8592529296875), angle=-82.80413055419922
Rect points: [[3190.980224609375, 2048.2919921875], [2644.459716796875, 1979.2906494140625], [2884.317138671875, 79.5234375], [3430.837646484375, 148.5247802734375]]


array([[3190.980, 2048.292],
       [2644.460, 1979.291],
       [2884.317, 79.523],
       [3430.838, 148.525]], dtype=float32)

In [23]:
for comb in all_indexes:
    rect1 = cv2.minAreaRect(all_contours[comb[0]])
    rect2 = cv2.minAreaRect(all_contours[comb[1]])
    rect_points1 = cv2.boxPoints(rect1)
    rect_points2 = cv2.boxPoints(rect2)
    print('{0} <-> {1}: {2:.2f} cm'.format(comb[0], comb[1], rect_dist(rect_points1, rect_points2)*real_image_ratio))

0 <-> 1: 6.45 cm
0 <-> 2: 8.22 cm
0 <-> 3: 1.90 cm
0 <-> 4: 14.21 cm
1 <-> 2: 5.19 cm
1 <-> 3: 2.93 cm
1 <-> 4: 2.49 cm
2 <-> 3: 7.94 cm
2 <-> 4: 5.01 cm
3 <-> 4: 9.63 cm


### Bonusová část

Pokud to pro vás byla doposuď zívačka, a chtěli byste nějaké body navíc, je možné získat až 2 prémiové body za trochu dalšího přemýšlení a programování. Za každý splněný úkol ze seznamu je __1 prémiový bod__. Maximálně je možné získat __až 2 prémiové__.

#### Úkoly:
1) Vytvořte vhodnou vizualizaci vzdáleností mezi objekty. Např. vykreslení nejkratší úsečky mezi obdélníky a vypsání její délky v centimetrech.

2) Upravte váš algoritmus tak, aby korektně fungoval i pro obrázek `patterns_photo_bonus.jpg`. Tedy podpora výpočtu nejkratší vzdálenosti mezi kombinací tvarů obdélník a kružnice. Jiné elipsové tvary pro jednoduchost neuvažujte. Může se vám na to hodit `cv2.fitEllipse()`.

3) Zkuste navrhnout jiné řešení než to využívající vzdálenost bodů od úsečky. Nemusí být 100% funkční ve všech případech, ale jasně vyspecifikujte jeho úskalí a případy kdy nebude fungovat.